In [18]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import random
%matplotlib inline

# Для работы с матрицами
from scipy.sparse import csr_matrix, coo_matrix

# Детерминированные алгоритмы
from implicit.nearest_neighbours import ItemItemRecommender, CosineRecommender, TFIDFRecommender, BM25Recommender

# Метрики
from implicit.evaluation import train_test_split
from implicit.evaluation import precision_at_k, mean_average_precision_at_k, AUC_at_k, ndcg_at_k

In [11]:
data = pd.read_csv('data/retail_train.csv')
data.head(2)

,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc
0,2375,26984851472,1,1004906,1,1.39,364,-0.6,1631,1,0.0,0.0
1,2375,26984851472,1,1033142,1,0.82,364,0.0,1631,1,0.0,0.0


In [12]:
test_size_weeks = 3

data_train = data[data['week_no'] < data['week_no'].max() - test_size_weeks]
data_test = data[data['week_no'] >= data['week_no'].max() - test_size_weeks]

In [15]:
result = data_train.groupby('user_id')['item_id'].unique().reset_index()
result.columns=['user_id', 'actual']

### Задание 1. Weighted Random Recommendation

Напишите код для случайных рекоммендаций, в которых вероятность рекомендовать товар прямо пропорциональна логарифму продаж
- Можно сэмплировать товары случайно, но пропорционально какому-либо весу
- Например, прямопропорционально популярности. Вес = log(sales_sum товара)

In [19]:
def weighted_random_recommendation(items_weights, n=5):
    """Случайные рекоммендации
    
    Input
    -----
    items_weights: pd.DataFrame
        Датафрейм со столбцами item_id, weight. Сумма weight по всем товарам = 1
    """
    """Топ-n популярных товаров"""
    
    # Подсказка: необходимо модифицировать функцию random_recommendation()
    items = np.array(items_weights['item_id'])
    weights = np.array(items_weights['weight'])
    recs = random.choices(items, weights=weights, k=n)

    return recs

In [20]:
%%time

items_weights = data_train.groupby('item_id')['sales_value'].sum().reset_index()
items_weights['weight'] = items_weights['sales_value'] / items_weights['sales_value'].sum()
items_weights = items_weights[['item_id', 'weight']]

result['weighted_random_recommendation'] = result['user_id'].apply(lambda x: weighted_random_recommendation(items_weights, n=5))

result.head(2)

CPU times: user 25.3 s, sys: 586 ms, total: 25.9 s
Wall time: 26.3 s


In [21]:
result.head(2)

,user_id,actual,weighted_random_recommendation
0,1,"[825123, 831447, 840361, 845307, 852014, 85498...","[823704, 6534178, 1022119, 1010578, 6533889]"
1,2,"[854852, 930118, 1077555, 1098066, 5567388, 55...","[2672344, 907945, 898394, 1102491, 6534178]"


## Задание 2. Улучшение бейзлайнов и ItemItem

- Попробуйте улучшить бейзлайны, считая случаный на топ-5000 товаров
- Попробуйте улучшить разные варианты ItemItemRecommender, выбирая число соседей $K$.

In [22]:
popularity = data_train.groupby('item_id')['quantity'].sum().reset_index()
popularity.rename(columns={'quantity': 'n_sold'}, inplace=True)

In [23]:
top_5000 = popularity.sort_values('n_sold', ascending=False).head(5000).item_id.tolist()

In [24]:
data_train.loc[~data_train['item_id'].isin(top_5000), 'item_id'] = 6666

/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:1817: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


### 1.1 Random recommendation

In [25]:
def random_recommendation(items, n=5):
    """Случайные рекоммендации"""
    
    items = np.array(items)
    recs = np.random.choice(items, size=n, replace=False)
    
    return recs.tolist()

In [26]:
%%time

items = data_train.item_id.unique()

result['random_recommendation_top_5000'] = result['user_id'].apply(lambda x: random_recommendation(items, n=5))

result.head(2)

CPU times: user 308 ms, sys: 1.64 ms, total: 310 ms
Wall time: 311 ms


In [27]:
result.head(2)

,user_id,actual,weighted_random_recommendation,random_recommendation_top_5000
0,1,"[825123, 831447, 840361, 845307, 852014, 85498...","[823704, 6534178, 1022119, 1010578, 6533889]","[947658, 826385, 9926758, 1043563, 945909]"
1,2,"[854852, 930118, 1077555, 1098066, 5567388, 55...","[2672344, 907945, 898394, 1102491, 6534178]","[857849, 12754036, 912553, 1070214, 8069118]"


### 1.2 Popularity-based recommendation

In [28]:
def popularity_recommendation(data, n=5):
    """Топ-n популярных товаров"""
    
    popular = data.groupby('item_id')['sales_value'].sum().reset_index()
    popular.sort_values('sales_value', ascending=False, inplace=True)
    
    recs = popular.head(n).item_id
    
    return recs.tolist()

In [29]:
%%time

popular_recs = popularity_recommendation(data_train, n=5)

result['popular_recommendation_top_5000'] = result['user_id'].apply(lambda x: popular_recs)
result.head(2)

CPU times: user 52.1 ms, sys: 969 µs, total: 53 ms
Wall time: 55.4 ms


In [30]:
result.head(2)

,user_id,actual,weighted_random_recommendation,random_recommendation_top_5000,popular_recommendation_top_5000
0,1,"[825123, 831447, 840361, 845307, 852014, 85498...","[823704, 6534178, 1022119, 1010578, 6533889]","[947658, 826385, 9926758, 1043563, 945909]","[6666, 6534178, 6533889, 1029743, 6534166]"
1,2,"[854852, 930118, 1077555, 1098066, 5567388, 55...","[2672344, 907945, 898394, 1102491, 6534178]","[857849, 12754036, 912553, 1070214, 8069118]","[6666, 6534178, 6533889, 1029743, 6534166]"


### 1.3 Weighted random recommender

In [31]:
def weighted_random_recommendation(items_weights, n=5):
    """Случайные рекоммендации
    
    Input
    -----
    items_weights: pd.DataFrame
        Датафрейм со столбцами item_id, weight. Сумма weight по всем товарам = 1
    """
    """Топ-n популярных товаров"""
    
    # Подсказка: необходимо модифицировать функцию random_recommendation()
    items = np.array(items_weights['item_id'])
    weights = np.array(items_weights['weight'])
    recs = random.choices(items, weights=weights, k=n)

    return recs

In [32]:
%%time

items_weights = data_train.groupby('item_id')['sales_value'].sum().reset_index()
items_weights['weight'] = items_weights['sales_value'] / items_weights['sales_value'].sum()
items_weights = items_weights[['item_id', 'weight']]

result['weighted_random_recommendation_top_5000'] = result['user_id'].apply(lambda x: weighted_random_recommendation(items_weights, n=5))

result.head(2)

CPU times: user 1.36 s, sys: 9.03 ms, total: 1.37 s
Wall time: 1.37 s


In [33]:
result.head(2)

,user_id,actual,weighted_random_recommendation,random_recommendation_top_5000,popular_recommendation_top_5000,weighted_random_recommendation_top_5000
0,1,"[825123, 831447, 840361, 845307, 852014, 85498...","[823704, 6534178, 1022119, 1010578, 6533889]","[947658, 826385, 9926758, 1043563, 945909]","[6666, 6534178, 6533889, 1029743, 6534166]","[5591038, 860037, 6534178, 6534178, 6666]"
1,2,"[854852, 930118, 1077555, 1098066, 5567388, 55...","[2672344, 907945, 898394, 1102491, 6534178]","[857849, 12754036, 912553, 1070214, 8069118]","[6666, 6534178, 6533889, 1029743, 6534166]","[854483, 962568, 8181555, 6666, 6666]"


### ItemItemRecommender

In [34]:
user_item_matrix = pd.pivot_table(data_train, 
                                  index='user_id', columns='item_id', 
                                  values='quantity',
                                  aggfunc='count', 
                                  fill_value=0
                                 )

user_item_matrix[user_item_matrix > 0] = 1 # так как в итоге хотим предсказать 

user_item_matrix = user_item_matrix.astype(float) # необходимый тип матрицы для implicit

# переведем в формат sparse matrix
sparse_user_item = csr_matrix(user_item_matrix)

In [35]:
userids = user_item_matrix.index.values
itemids = user_item_matrix.columns.values

matrix_userids = np.arange(len(userids))
matrix_itemids = np.arange(len(itemids))

id_to_itemid = dict(zip(matrix_itemids, itemids))
id_to_userid = dict(zip(matrix_userids, userids))

itemid_to_id = dict(zip(itemids, matrix_itemids))
userid_to_id = dict(zip(userids, matrix_userids))

In [36]:
def item_item_recommender_different_k(result, id_to_itemid, id_to_userid, itemid_to_id, userid_to_id, k):
    
    col_name = 'itemitem_k_'
    
    for i in range(1, k+1):
        
        col_name_ = col_name + str(i)

        model = ItemItemRecommender(K=i, num_threads=4) # K - кол-во билжайших соседей

        model.fit(csr_matrix(user_item_matrix).T.tocsr(),  # На вход item-user matrix
              show_progress=True)

        recs = model.recommend(userid=userid_to_id[2],  # userid - id от 0 до N
                            user_items=csr_matrix(user_item_matrix).tocsr(),   # на вход user-item matrix
                            N=5, # кол-во рекомендаций 
                            filter_already_liked_items=False, 
                            filter_items=None, 
                            recalculate_user=True)
        
        result[col_name_] = result['user_id'].apply(lambda user_id: [id_to_itemid[rec[0]]  
                                                                  for rec in model.recommend(userid=userid_to_id[user_id], 
                                                                    user_items=sparse_user_item,   # на вход user-item matrix
                                                                    N=5, 
                                                                    filter_already_liked_items=False, 
                                                                    filter_items=None, 
                                                                    recalculate_user=True)
                                                                                 ])
    return result

In [37]:
# filter_items=[itemid_to_id[6666]], 

In [38]:
k=5
result = item_item_recommender_different_k(result, id_to_itemid, id_to_userid, itemid_to_id, userid_to_id, k)

  0%|          | 0/5001 [00:00<?, ?it/s]

  0%|          | 0/5001 [00:00<?, ?it/s]

  0%|          | 0/5001 [00:00<?, ?it/s]

  0%|          | 0/5001 [00:00<?, ?it/s]

  0%|          | 0/5001 [00:00<?, ?it/s]

In [39]:
result.head(2)

,user_id,actual,weighted_random_recommendation,random_recommendation_top_5000,popular_recommendation_top_5000,weighted_random_recommendation_top_5000,itemitem_k_1,itemitem_k_2,itemitem_k_3,itemitem_k_4,itemitem_k_5
0,1,"[825123, 831447, 840361, 845307, 852014, 85498...","[823704, 6534178, 1022119, 1010578, 6533889]","[947658, 826385, 9926758, 1043563, 945909]","[6666, 6534178, 6533889, 1029743, 6534166]","[5591038, 860037, 6534178, 6534178, 6666]","[6666, 1082185, 995242, 1029743, 840361]","[6666, 1082185, 995242, 1029743, 840361]","[6666, 1082185, 981760, 995242, 1029743]","[6666, 1082185, 981760, 995242, 1127831]","[6666, 1082185, 981760, 1127831, 995242]"
1,2,"[854852, 930118, 1077555, 1098066, 5567388, 55...","[2672344, 907945, 898394, 1102491, 6534178]","[857849, 12754036, 912553, 1070214, 8069118]","[6666, 6534178, 6533889, 1029743, 6534166]","[854483, 962568, 8181555, 6666, 6666]","[6666, 1082185, 840361, 1098066, 826249]","[6666, 1082185, 840361, 1098066, 826249]","[6666, 1082185, 981760, 840361, 1098066]","[6666, 1082185, 981760, 1127831, 995242]","[6666, 1082185, 981760, 1127831, 1098066]"


In [40]:
def recall(recommended_list, bought_list):
    
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)
    
    flags = np.isin(bought_list, recommended_list)
    return flags.sum() / len(bought_list)
    

def recall_at_k(recommended_list, bought_list, k=5):
    return recall(recommended_list[:k], bought_list)


def precision(recommended_list, bought_list):
    
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)
    
    flags = np.isin(bought_list, recommended_list)
    return flags.sum() / len(recommended_list)


def precision_at_k(recommended_list, bought_list, k=5):
    return precision(recommended_list[:k], bought_list)

In [41]:
for name_col in result.columns[1:]:
    print(f"{round(result.apply(lambda row: precision_at_k(row[name_col], row['actual']), axis=1).mean(),4)}:{name_col}")

1.0:actual
0.1022:weighted_random_recommendation
0.0528:random_recommendation_top_5000
0.2318:popular_recommendation_top_5000
0.0954:weighted_random_recommendation_top_5000
0.7947:itemitem_k_1
0.7573:itemitem_k_2
0.6834:itemitem_k_3
0.5778:itemitem_k_4
0.5437:itemitem_k_5
